In [16]:
import pandas as pd
import numpy as np
from tqdm import tqdm

In [8]:
df = pd.read_csv('d2_ligands_docked.csv')

0          COC1=NOCC=C1C(O)C2=CC=NC=C2N3CC=CC=C3OCC=C
1                         CCNS(=O)(=O)CCC1=CC=CC=C1CC
2           OC(C)C=NC=C(N1C=NC(C2=CC=CC=N2)=N1)C(CF)F
3                          C=1NCCCC=1S(=O)(C)CCC(CF)F
4                    CCCS(=O)(=O)C1=CC=CC=C1N2C=NCC2S
                            ...                      
995                     C=CC=C1C(C=CC=C1)C2=CC=CC=C2O
996                            CC=CC1=CC=CC(=C1)OCC=O
997    CCCC1=NC=CC=C1S(=O)(=O)N2C=NC3=CC(F)=CC(F)=C32
998                        O=C(C=CC1=CC=CC=C1NC=C)CBr
999                           C=CCN(CC(=O)C)C1CCCC1=N
Name: smiles, Length: 1000, dtype: object

In [15]:
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit import DataStructs

def get_max_train_similarity(mol):
    train = pd.read_parquet('../data/train_data/big_dataset_std.parquet')
    tanimotos = []
    for smile in tqdm(train['smiles'].values):
        train_mol = Chem.MolFromSmiles(smile)
        train_fp = AllChem.GetMorganFingerprintAsBitVect(train_mol, 2, nBits=512)
        query_mol = Chem.MolFromSmiles(smile)
        query_fp = AllChem.GetMorganFingerprintAsBitVect(query_mol, 2, nBits=512)
        sim = DataStructs.TanimotoSimilarity(train_fp, query_fp)
        tanimotos.append(sim)
    return max(tanimotos)

In [ ]:
test_smile = 'COC1=NOCC=C1C(O)C2=CC=NC=C2N3CC=CC=C3OCC=C'
test_mol = Chem.MolFromSmiles(test_smile)
get_max_train_similarity(test_mol)